In [2]:
# %%capture captured_output
# 实验环境已经预装了mindspore==2.3.0，如需更换mindspore版本，可更改下面 MINDSPORE_VERSION 变量
!pip uninstall mindspore -y
!export MINDSPORE_VERSION=2.3.0
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/${MINDSPORE_VERSION}/MindSpore/unified/aarch64/mindspore-${MINDSPORE_VERSION}-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.mirrors.ustc.edu.cn/simple

!pip install https://repo.mindspore.cn/mindspore-lab/mindnlp/newest/any/mindnlp-0.4.0-py3-none-any.whl -i https://pypi.mirrors.ustc.edu.cn/simple
!pip install jieba
%env HF_ENDPOINT=https://hf-mirror.com

Found existing installation: mindspore 2.3.0
Uninstalling mindspore-2.3.0:
  Successfully uninstalled mindspore-2.3.0
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
  Using cached https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.3.0/MindSpore/unified/aarch64/mindspore-2.3.0-cp39-cp39-linux_aarch64.whl (273.2 MB)
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
  Using cached https://repo.mindspore.cn/mindspore-lab/mindnlp/newest/any/mindnlp-0.4.0-py3-none-any.whl (8.3 MB)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
env: HF_ENDPOINT=https://hf-mirror.com


In [ ]:
wget https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/model/ZhipuAI/glm-4-9b-chat.zip
mkdir -p ./.mindnlp/model/ZhipuAI
unzip -d ./.mindnlp/model/ZhipuAI/ glm-4-9b-chat.zip

In [3]:
from mindnlp.peft import (
    get_peft_config,
    get_peft_model,
    LoraConfig,
    PeftType,
    PeftConfig,
    PeftModel,
)
import argparse
import os

import mindspore
from mindspore.common.api import _no_grad

from tqdm import tqdm
from mindnlp.core.optim import AdamW
from mindnlp import evaluate
from mindnlp.dataset import load_dataset
from mindnlp.engine import set_seed
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup
from mindnlp.peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    LoraConfig,
)

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the default dict

In [4]:
import mindspore
from mindnlp.core import no_grad
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindspore._c_expression import _framework_profiler_step_start
from mindspore._c_expression import _framework_profiler_step_end
# 加载tokenizer文件

In [5]:
batch_size = 32
model_name_or_path = "./.mindnlp/model/ZhipuAI/glm-4-9b-chat"
task = "mrpc"
peft_type = PeftType.PROMPT_TUNING
num_epochs = 20
peft_config = LoraConfig(task_type="QUESTION_ANS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1, target_cells)
lr = 3e-4

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, mirror='modelscope')

# # 在此输入希望和ChatGLM4对话的内容
# query = "你好"

# inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
#                                        add_generation_prompt=True,
#                                        tokenize=True,
#                                        return_tensors="ms",
#                                        return_dict=True
#                                        )
# print(inputs)
# 加载模型

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    mirror='modelscope',
    ms_dtype=mindspore.float16,
).eval()


if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
model = get_peft_model(model, peft_config)
# print(model)
model.print_trainable_parameters()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

AttributeError: 'LoraConfig' object has no attribute 'target_cells'

In [9]:
classes = [
    "耳鼻喉科",
    "肿瘤疾病",
    "肛肠",
    "泌尿科",
    "血管瘤",
    "肺癌",
    "神经脑外科",
    "甲状腺腺瘤",
    "普通外科",
    "乳腺癌",
    "肿瘤与疼痛",
    "乳腺科",
    "宫颈癌",
    "小儿血管瘤",
    "肝胆科",
    "淋巴瘤",
    "肝癌",
    "营养保健科",
    "甲状腺癌",
    "心外科",
    "血管科",
    "胃癌",
    "卵巢癌",
    "软纤维瘤",
    "口腔癌",
    "鼻咽癌",
    "结肠癌",
    "胰腺癌",
    "子宫癌",
    "胆管癌",
    "前列腺癌",
    "骨癌",
    "淋巴癌",
    "肾癌",
    "直肠癌",
    "乳腺纤维腺瘤",
    "脑瘤",
    "黑色素瘤",
    "小儿肿瘤",
    "脂肪瘤",
    "新生儿科",
    "神经内科",
    "膀胱癌",
    "大肠癌",
    "外阴癌",
    "口腔颌面肿瘤",
    "淋巴管瘤",
    "蛛网膜囊肿",
    "皮肤纤维瘤"
]


In [28]:
import mindspore.dataset as ds
csv_dataset_dir = ["dataset/data.csv"] # contains 1 or multiple csv files
dataset = ds.CSVDataset(dataset_files=csv_dataset_dir, column_names=['ask', 'answer', 'department'])
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])
next(train_dataset.create_dict_iterator())
# def add_text_label(ask, answer, label):
#     return ask, answer, classes[label.item()]

# train_dataset = train_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])
# validation_dataset = validation_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])

# def add_text_label(ask, label):
#     return ask, label, classes[label.item()]

# train_dataset = train_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])
# validation_dataset = validation_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])

NameError: name 'string' is not defined

In [26]:
next(train_dataset.create_dict_iterator())

[ERROR] MD(1246,fffe57f7f120,python):2024-08-28-11:21:04.642.892 [mindspore/ccsrc/minddata/dataset/util/task_manager.cc:230] InterruptMaster] MindSpore dataset is terminated with err msg: Exception thrown from dataset pipeline. Refer to 'Dataset Pipeline Error Message'. Invalid file, failed to parse csv file: dataset/data.csv at line 1. Error message: Invalid columns, the size of column_names should be less than the size of 'column_defaults', but got the size of column_names: 2, the size of 'column_defaults': 3.
Line of code : 507
File         : mindspore/ccsrc/minddata/dataset/engine/datasetops/source/csv_op.cc



RuntimeError: Exception thrown from dataset pipeline. Refer to 'Dataset Pipeline Error Message'. 

------------------------------------------------------------------
- Dataset Pipeline Error Message: 
------------------------------------------------------------------
[ERROR] Invalid file, failed to parse csv file: dataset/data.csv at line 1. Error message: Invalid columns, the size of column_names should be less than the size of 'column_defaults', but got the size of column_names: 2, the size of 'column_defaults': 3.

------------------------------------------------------------------
- C++ Call Stack: (For framework developers) 
------------------------------------------------------------------
mindspore/ccsrc/minddata/dataset/engine/datasetops/source/csv_op.cc(507).




In [ ]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=3, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)